In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import re

sys.path.append("../")
warnings.filterwarnings("ignore")

In [2]:
import os


root_dir = os.path.abspath('../bailian_nlp/')

data_path = os.path.join(root_dir, 'datadrive/bailian/pos')
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

train_path = os.path.join(data_path, 'train.csv')
valid_path = os.path.join(data_path, 'valid.csv')

model_dir = os.path.join(root_dir, 'datadrive/models/chinese_L-12_H-768_A-12/')
init_checkpoint_pt = os.path.join(model_dir, 'bert_model.bin')
bert_config_file = os.path.join(model_dir, 'bert_config.json')
vocab_file = os.path.join(model_dir, 'vocab.txt')
model_pt = os.path.join(model_dir, 'pos.bin')
config_file = os.path.join(model_dir, 'pos.json')



In [38]:
# 初始化构造数据
from bailian_nlp.preprocessing.build_data import build_pos_fake_data, build_pos_train_and_valid_data
# build_pos_fake_data()
build_pos_train_and_valid_data()



195 21700 0.008986175115207374
274 31200 0.008782051282051282
276 31700 0.008706624605678233
307 36100 0.00850415512465374
327 36600 0.008934426229508196
356 37500 0.009493333333333333
435 46200 0.009415584415584415
624 67800 0.009203539823008849
724 76800 0.009427083333333334
914 98800 0.009251012145748989
1091 120300 0.009068994181213633
1204 134500 0.008951672862453531
1397 156900 0.008903760356915232
1415 158700 0.008916194076874606
1725 203000 0.008497536945812808
2048 416200 0.004920711196540125
2054 793378 0.002588929866973876


In [ ]:
# 正常训练

# from bailian_nlp.modules import BertData

# data = BertData.create(
#     train_path,
#     valid_path, 
#     vocab_file,
#     data_type="bert_uncased",
#     max_seq_len=128,
#     batch_size=128
    
# )

import torch
import torch.nn as nn
from importlib import reload
from bailian_nlp.modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    bert_config_file, 
    init_checkpoint_pt,
    enc_hidden_dim=256
)


from bailian_nlp.modules.train import train
reload(train)
num_epochs = 15
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')


In [39]:
num_epochs = 1

learner.fit(num_epochs, target_metric='f1')

INFO:root:Resuming train... Current epoch 16.


In [40]:
# 恢复训练

from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()
tagger.init_env(for_train=True)

data = tagger.learner.data
learner = tagger.learner
num_epochs = 1
# learner.load_model()
learner.t_total = num_epochs * len(data.train_dl)
learner.sup_labels = list(set(data.id2label[1:]) | set(learner.sup_labels))
learner.fit(num_epochs, target_metric='f1')


INFO:bailian_nlp.released.pos:load default user_dict in /home/liuxiang/Projects/pos-bert/bailian_nlp/datadrive/dict/user_dict.txt
INFO:bailian_nlp.released.dictionary:本次加载词条数：3
INFO:bailian_nlp.released.dictionary:当前总词条数: 3
INFO:root:Loading pretrained bert model!
INFO:root:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

INFO:bailian_nlp.released.pos:found pos model file in /home/liuxiang/Projects/pos-bert/bailian_nlp/datadrive/models/chinese_L-12_H-768_A-12/pos.bin
INFO:bailian_nlp.released.pos:pos model loads success!


DEBUG:bailian_nlp.web.utils.common:get_data cost 82.997559s


DEBUG:bailian_nlp.web.utils.common:get_data cost 0.203217s
INFO:root:Loading pretrained bert model!
INFO:root:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}



INFO:bailian_nlp.released.pos:found pos model file in /home/liuxiang/Projects/pos-bert/bailian_nlp/datadrive/models/chinese_L-12_H-768_A-12/pos.bin
INFO:bailian_nlp.released.pos:pos model loads success!
INFO:root:Resuming train... Current epoch 0.


INFO:root:
epoch 1, average train epoch loss=1.9717



INFO:root:on epoch 0 by max_f1: 0.951
INFO:root:Saving new best model...


              precision    recall  f1-score   support

         S_r      0.993     0.983     0.988       414
         I_m      0.950     0.975     0.962       589
        E_an      0.760     0.745     0.752        51
        B_xc      0.714     0.714     0.714        14
         E_c      0.979     0.960     0.969       247
        I_vn      0.916     0.826     0.869       172
         I_d      0.972     0.921     0.946        76
         S_f      0.980     0.974     0.977       497
        I_nw      0.839     0.833     0.836       162
        S_ad      0.944     0.708     0.810        24
        S_ns      0.667     1.000     0.800         2
        B_nt      0.958     0.976     0.967      4949
        E_nw      0.909     0.820     0.862        61
         S_c      0.975     0.979     0.977       845
        I_xc      0.600     0.600     0.600         5
         S_q      0.929     0.890     0.909        73
         I_s      1.000     0.750     0.857         8
         S_p      0.964    

In [ ]:
learner.fit(num_epochs, target_metric='f1')

In [ ]:
from modules.data import bert_data
reload(bert_data)
dl = bert_data.get_bert_data_loader_for_predict(valid_path, learner)

In [ ]:
# learner.load_model()
preds = learner.predict(dl)

In [ ]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds)
print(clf_report)

In [45]:
from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()


INFO:bailian_nlp.released.pos:load default user_dict in /home/liuxiang/Projects/pos-bert/bailian_nlp/datadrive/dict/user_dict.txt
INFO:bailian_nlp.released.dictionary:本次加载词条数：3
INFO:bailian_nlp.released.dictionary:当前总词条数: 3


TypeError: from_config() got an unexpected keyword argument 'bert_config_file'

In [44]:
import time

st = time.time()
text = '近日，编程猫（深圳点猫科技有限公司）正式对外宣布完成B轮1.2亿元融资。本轮融资由高瓴资本领投，清流资本、清晗基金跟投，天使轮投资者猎豹移动继续跟投。'
# text = '未来编程教育产业将蓬勃发展，编程猫作为提供工具与内容的企业，有望长期处于行业领跑者地位。'
# text = '美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司始创于2004年,是中国健康体检和医疗服务集团,总部位于上海,深耕布局北京、深圳、沈阳、广州、成都、武汉、...'
# text = '百炼智能百炼智能'
# text = '高越君冯是聪'
# text = '周光明确否认CEO佟显侨和衡量推动发出公司公告'
# text = '周光明确否认CEO佟显侨和CTO衡量说的罪名'
# text = '董事'
text = '一言不合'
# text = '客户包括雀巢、洲际酒店、瑞士航空、德意志银行、红牛、瑞士联合银行等世界知名公司。'
# text = '药方越是多的，越表明病是难的于治疗'
# text = '2012-11-8 的客户包括：宝洁 、杜邦、强生、百事可 乐、3M 、三菱、吉列、通用医疗系统、美国 运通、雅芳、纽约银 行、百时美—施贵 宝、礼来、迪斯尼、纳贝斯克、纽约'
# text = '网：tmall.com'
res = learner.cut(text)
ed = time.time()
print(ed - st)
res

AttributeError: 'NerLearner' object has no attribute 'cut'

In [24]:
a = 'á'
acc = a[1]

In [67]:
tokenizer.tokenize(ww)

(['1',
  '.',
  '未',
  'a',
  '来',
  '伙',
  '伴',
  '机',
  '器',
  '人',
  '费',
  '旭',
  '锋',
  '：',
  '教',
  '育',
  '机',
  '器',
  '人',
  '市',
  '场',
  '预',
  '计',
  '在',
  '2025',
  '年',
  '会',
  '达',
  '到',
  '3000',
  '亿'],
 [(-2, (1, 2)),
  (-2, (3, 4)),
  (-2, (4, 5)),
  (1, (0, 1)),
  (1, (1, 2)),
  (1, (5, 6)),
  (-2, (7, 8)),
  (1, (6, 7)),
  (1, (8, 9)),
  (-2, (9, 10)),
  (-2, (10, 11)),
  (1, (11, 12)),
  (1, (12, 13)),
  (1, (13, 14)),
  (1, (14, 15)),
  (1, (15, 16)),
  (1, (16, 17)),
  (1, (17, 18)),
  (1, (18, 19)),
  (1, (19, 20)),
  (0, (20, 21)),
  (1, (21, 22)),
  (1, (22, 23)),
  (1, (23, 24)),
  (1, (24, 25)),
  (1, (25, 26)),
  (1, (26, 27)),
  (1, (27, 28)),
  (1, (28, 29)),
  (1, (29, 30)),
  (1, (30, 31)),
  (1, (31, 35)),
  (1, (35, 36)),
  (1, (36, 37)),
  (1, (37, 38)),
  (1, (38, 39)),
  (1, (39, 43)),
  (1, (43, 44))])

In [66]:
ww = '1́.́́未á来́́伙伴机器人费旭锋： 教育机器人市场预计在2025年会达到3000亿'
ww[5]

'未'

In [69]:
from bailian_nlp.modules.data.tokenization import BailianTokenizer
tokenizer = BailianTokenizer()
text = f'1{acc}/m ./w {acc}{acc}未á来{acc}{acc}伙伴机器人/nt 费旭锋/nr ：/w  /w 教育机器人/nz 市场/n 预计/v 在/p 2025/m 年会/n 达到/v 3000亿/m'
tokenizer.tokenize_with_pos_text(text)

1 S_m
. S_w
未 B_nt
a I_nt
来 I_nt
伙 I_nt
伴 I_nt
机 I_nt
器 I_nt
人 E_nt
费 B_nr
旭 I_nr
锋 E_nr
： S_w
教 B_nz
育 I_nz
机 I_nz
器 I_nz
人 E_nz
市 B_n
场 E_n
预 B_v
计 E_v
在 S_p
2025 S_m
年 B_n
会 E_n
达 B_v
到 E_v
3000 B_m
亿 E_m


(['1',
  '.',
  '未',
  'a',
  '来',
  '伙',
  '伴',
  '机',
  '器',
  '人',
  '费',
  '旭',
  '锋',
  '：',
  '教',
  '育',
  '机',
  '器',
  '人',
  '市',
  '场',
  '预',
  '计',
  '在',
  '2025',
  '年',
  '会',
  '达',
  '到',
  '3000',
  '亿'],
 ['S_m',
  'S_w',
  'B_nt',
  'I_nt',
  'I_nt',
  'I_nt',
  'I_nt',
  'I_nt',
  'I_nt',
  'E_nt',
  'B_nr',
  'I_nr',
  'E_nr',
  'S_w',
  'B_nz',
  'I_nz',
  'I_nz',
  'I_nz',
  'E_nz',
  'B_n',
  'E_n',
  'B_v',
  'E_v',
  'S_p',
  'S_m',
  'B_n',
  'E_n',
  'B_v',
  'E_v',
  'B_m',
  'E_m'])

In [20]:
a[6]

'/'